<a href="https://colab.research.google.com/github/MariDamk/cnn-transfer-learning-cats-dogs/blob/main/CNN_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CNN Cats vs Dogs Image Classification #

##Changed to runtime type to T4 GPU instead of CPU, for faster results
## Importing necessary libraries
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


## Defining temporary path and downloading the dataset with wget because for some reason with tf.keras.utils.get_file there was FileNotFound Error
data_path = '/tmp/cats_and_dogs_filtered.zip'
extract_path = '/tmp/'

print("Downloading dataset... Chill...")
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O {data_path}

## Unziping the file
with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

## Directory paths
base_dir = os.path.join(extract_path, 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


IMG_SIZE = 128
BATCH_SIZE = 32

## Modified versions of the images are created to prevnt overfitting
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary' ## cat or dog
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


# Using MobileNetV2
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


## Training the model with just 15 epochs
EPOCHS = 15
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

## plotting
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    sns.set_style("whitegrid")
    plt.figure(figsize=(16, 6))

## Accuracy Plotting
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'o-', label='Training Accuracy', color='darkorange', markersize=5)
    plt.plot(epochs, val_acc, 'o-', label='Validation Accuracy', color='steelblue', markersize=5)
    plt.title('Model Accuracy', fontsize=16)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(loc='lower right', fontsize=11)
    plt.ylim([min(plt.ylim())-0.05, 1.05])

## Loss Plotting
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'o-', label='Training Loss', color='darkorange', markersize=5)
    plt.plot(epochs, val_loss, 'o-', label='Validation Loss', color='steelblue', markersize=5)
    plt.title('Model Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(loc='upper right', fontsize=11)
    plt.ylim([0, max(plt.ylim())+0.1])

    plt.suptitle('Enhanced Training & Validation History', fontsize=20, y=1.02)
    plt.tight_layout()
    plt.show()

plot_history(history)